In [1]:
from pathlib import Path
from typing import List
import gzip
import numpy as np
import pandas as pd

import torch.utils.data as Data 
import torch
import matplotlib.pyplot as plt
from torch import nn ,optim

In [2]:
if torch.cuda.is_available():
    device=torch.device('cuda')
    print("GPU获取")

GPU获取


In [4]:
data_participants_dir='E:/Data/participants1.tsv'#获取全部实验者的数据
data_participants_tsv=pd.read_csv(data_participants_dir,sep='\t')
data_NoNaN=data_participants_tsv.dropna(axis=0,how='any')#去除NAN无效数据
data_sex=data_NoNaN.sex.head(100)
data_age=data_NoNaN.age.head(100)
data_sex_code=[]
data_sex

0      F
1      F
2      M
3      F
4      M
      ..
100    F
101    M
102    F
103    F
104    F
Name: sex, Length: 100, dtype: object

In [5]:
for i in data_sex:#将女性设置为1，男性设置为0
    gender=[]
    if(i=="F"):
        data_sex_code.append(1)
    else:
        data_sex_code.append(0)
    
data_sex_code

[1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1]

In [6]:
data_gender=np.array(data_sex_code)
type(data_gender)


numpy.ndarray

In [7]:
spatial_map_trian = np.load('E:/Data/PIOP1/input/spatial_map_train.npy')#获取样本数据
len(spatial_map_trian)


100

In [8]:
train_target=torch.tensor(data_gender,dtype=torch.float)
spatial_map_trian=torch.tensor(spatial_map_trian, dtype=torch.float)#转化成tensor类别

In [9]:
batch_size=10
dataset = Data.TensorDataset(spatial_map_trian, train_target)
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)#创建DataLoader 方便读取数据
for X,y in data_iter:
    print(X.shape,len(y))
    

torch.Size([10, 240, 240, 220]) 10
torch.Size([10, 240, 240, 220]) 10
torch.Size([10, 240, 240, 220]) 10
torch.Size([10, 240, 240, 220]) 10
torch.Size([10, 240, 240, 220]) 10
torch.Size([10, 240, 240, 220]) 10
torch.Size([10, 240, 240, 220]) 10
torch.Size([10, 240, 240, 220]) 10
torch.Size([10, 240, 240, 220]) 10
torch.Size([10, 240, 240, 220]) 10


In [67]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet,self).__init__()
        self.conv=nn.Sequential(
            nn.Conv3d(1,4,12,4),#240*240*240->58*58*58
            nn.BatchNorm3d(4),
            nn.ReLU(),
            nn.MaxPool3d(2,2),#58*58*58->29*29*29
            nn.Conv3d(4,8,3,1,1),#29*29*29->29*29*29
            nn.BatchNorm3d(8),
            nn.ReLU(),
            nn.MaxPool3d(2,2),#29*29*29->14*14*14
            nn.Conv3d(8,16,3,1,1),#14*14*14->14*14*14
            nn.BatchNorm3d(16),
            nn.ReLU(),
            nn.MaxPool3d(2,2),#14*14*14->7*7*7
            nn.Conv3d(16,32,3,1,1),#7*7*7->7*7*7
            nn.BatchNorm3d(32),
            nn.ReLU(),
            nn.MaxPool3d(2,2)#7*7*7->3*3*3
            
        )
        self.fc=nn.Sequential(
            nn.Linear(32*3*3*3,512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512,64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64,2)#预测性别  
         )
    def forward (self,img):
        padding=nn.ConstantPad3d((10,10,0,0,0,0),0.0)
        img=padding(img)
        batch_size, x, y, z = img.shape
        img = img.reshape(batch_size, 1, x, y, z)
        feature=self.conv(img)
        output=self.fc(feature.view(img.shape[0],-1))
        return output

net=AlexNet()
x=torch.zeros(1,240,240,220)
output=net(x)
output.shape

torch.Size([1, 2])

In [ ]:
net=net.to(device)
loss=nn.CrossEntropyLoss()
result=[]
lr=0.001
num_epoches=20
optimizer=torch.optim.Adam(net.parameters(),lr=lr)
for epoch in range(num_epoches):
    train_l_sum, train_acc_sum, n=0.0,0.0,0
    for X,y in data_iter:
        X=X.to(device)
        y=y.to(device)
        y=y.long()
        y_hat=net(X)
        l=loss(y_hat,y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        train_l_sum+=l.cpu().item()
        n+=y.shape[0]
        train_acc_sum += (y_hat.argmax(dim=1) ==y).sum().cpu().item()
    
    result.append(train_acc_sum / n)
    print('epoch %d, loss %.4f,  train acc %.3f'%(epoch + 1, train_l_sum / n,train_acc_sum / n))